In [1]:
!gdown --id 1cXgMoIbKIgLjOAZssEei9P1IjGqjSvKa

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1cXgMoIbKIgLjOAZssEei9P1IjGqjSvKa
To: /content/boston.csv
100% 39.2k/39.2k [00:00<00:00, 45.7MB/s]


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [3]:
df = pd.read_csv('/content/boston.csv')
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [4]:
x = df.drop(['PRICE'],axis=1)
y = df['PRICE']

In [5]:
scaler1 = StandardScaler()
x = scaler1.fit_transform(x)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=True)

In [7]:
y_train = np.expand_dims(y_train.values,axis=1)
y_test = np.expand_dims(y_test.values,axis=1)

In [8]:
x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)
y_train = torch.from_numpy(y_train).type(torch.FloatTensor)
y_test = torch.from_numpy(y_test).type(torch.FloatTensor)

In [9]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(13, 26)
        self.fc2 = nn.Linear(26, 52)
        self.fc3 = nn.Linear(52, 104)
        self.fc4 = nn.Linear(104, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x
    
model = Model()

In [10]:
loss = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-5)

In [11]:
epochs = 125000

for i in range(epochs + 1):
    optimizer.zero_grad()
    y_pred = model(x_train)
    
    cost = loss(y_pred, y_train)
    cost.backward()
    optimizer.step()
    
    if i % 10000 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            i, epochs, cost.item()
        ))

Epoch    0/125000 Cost: 583.650269
Epoch 10000/125000 Cost: 26.685280
Epoch 20000/125000 Cost: 8.353229
Epoch 30000/125000 Cost: 3.272939
Epoch 40000/125000 Cost: 1.368474
Epoch 50000/125000 Cost: 0.624440
Epoch 60000/125000 Cost: 0.235172
Epoch 70000/125000 Cost: 0.083595
Epoch 80000/125000 Cost: 0.031836
Epoch 90000/125000 Cost: 0.010169
Epoch 100000/125000 Cost: 0.003435
Epoch 110000/125000 Cost: 0.001255
Epoch 120000/125000 Cost: 0.000492


In [12]:
model.eval()
with torch.no_grad():
    outputs = model(x_test)
    cost = loss(outputs, y_test)
    print('test loss: %.4f' % cost.item())

test loss: 37.1401
